In [1]:
# scores_teamhydrogen-white-malamute-prompt-openorca-v2_test-cirrus-e5-large-title
# 0.91941

# scores_wealthy-ant_test-e5-base-title
# 0.924605

# scores_teamhydrogen-cool-giraffe-prompt-2409_test-gte-large-title
# 0.92579

# scores_teamhydrogen-nostalgic-mammoth-avg_test-cirrus-gte-base-title-3
# 0.92602

# scores_teamhydrogen-raspberry-beaver-1_test-cirrus-e5-large-title
# 0.92659

# scores_teamhydrogen-cool-giraffe-prompt-0710_test-cirrus-bge-large-title-ind
# 0.92656

In [2]:
from datasets import load_dataset
from tqdm.notebook import tqdm
import torch
import pandas as pd
from glob import glob
import numpy as np
import os
os.environ["TRANSFORMERS_OFFLINE"] = "1"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [3]:
!pip install /kaggle/input/sci-llm-pip-v2/sentence-transformers-2.2.2.tar.gz

Processing /kaggle/input/sci-llm-pip-v2/sentence-transformers-2.2.2.tar.gz
  Preparing metadata (setup.py) ... - \ done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=16e873074ef568e3b149003dd1304d52e366da97bd0c082ea14de65ff2a2ed82
  Stored in directory: /root/.cache/pip/wheels/c6/26/e2/13ef17530724efc5be0bf3d290de1ecaa6c0ff0225fd548014
Successfully built sentence-transformers


In [4]:
from pathlib import Path

data_path = Path('/kaggle/input/kaggle-llm-science-exam')


if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    test = pd.read_csv(data_path / 'test.csv')
    CALC_SCORE = False
else:
    test = pd.read_csv("/kaggle/input/dataset-wiki-new-1/dataset_wiki_new_1_balanced.csv")
    test["id"] = np.arange(len(test))
#     test = pd.read_csv(data_path / 'train.csv')
    CALC_SCORE = True
test.head()

,prompt,A,B,C,D,E,answer,id
0,What is the method of transcription in the lif...,RNA-templated transcription is the method of t...,Transcription occurs through a unique mechanis...,Reverse transcription is the method of transcr...,DNA-templated transcription is the method of t...,Transcription does not occur in the life cycle...,D,0
1,What is the role of the viral fiber glycoprote...,The viral fiber glycoproteins are involved in ...,The viral fiber glycoproteins code for 40 prot...,The viral fiber glycoproteins are responsible ...,The viral fiber glycoproteins mediate endocyto...,The viral fiber glycoproteins are responsible ...,D,1
2,What is the significance of the faint Hα emiss...,The emission lines indicate that 3 Geminorum i...,The emission lines indicate that 3 Geminorum i...,The emission lines indicate that 3 Geminorum i...,The emission lines indicate that 3 Geminorum i...,The emission lines indicate that 3 Geminorum i...,A,2
3,What is the significance of the pedicellariae ...,They are used for climbing on corals.,They resemble the traps of the Venus fly trap ...,They are covered by short and stout spines.,They are found on the central disc of the sea ...,They are a characteristic feature of the Gonia...,B,3
4,What is the role of the microprocessor complex...,The microprocessor complex is responsible for ...,The microprocessor complex is responsible for ...,The microprocessor complex is involved in the ...,The microprocessor complex is involved in the ...,The microprocessor complex is responsible for ...,A,4


In [5]:
test.to_parquet("test_raw.pq")

In [6]:
%%writefile get_topk.py

import os
import pandas as pd
from glob import glob
import torch
import numpy as np
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
from pathlib import Path
from joblib import Parallel, delayed
import argparse

if __name__ == "__main__":
    
    ap = argparse.ArgumentParser()
    ap.add_argument('--wiki', type=str, required=True)
    ap.add_argument('--model_name', type=str, required=True)
    ap.add_argument('--test_file', type=str, required=True)
    ap.add_argument('--topk', type=int, required=True)
    ap.add_argument('--ind', type=int, required=True)
    args = ap.parse_args()

    if args.ind == 1:
        TOP_K = 5 * 5
    else:
        if args.topk == 10:
            TOP_K = 20
        else:
            TOP_K = 10

    data_path = Path('/kaggle/input/kaggle-llm-science-exam')
    
    if args.wiki == "cirrus":
        print("Using cirrus wiki")
        files_all = sorted(list(glob("/kaggle/input/cirruswiki-titles/*.parquet")))
    elif args.wiki == "new":
        print("Using new wiki")
        files_all = sorted(list(glob("/kaggle/input/newwiki-titles/*.parquet")))
    elif args.wiki == "last":
        print("Using last wiki")
        files_all = sorted(list(glob("/kaggle/input/cirruswiki-256-titles/*.parquet"))) 
        
    if args.wiki == "cirrus":
        if "e5-base" in args.model_name:
            files_np = sorted(list(glob("/kaggle/input/cirruswiki-e5-part?/*.npy")))
            files_np += sorted(list(glob("/kaggle/input/cirruswiki-e5-part9-1/*.npy")))
        elif "e5-large" in args.model_name:
            files_np = sorted(list(glob("/kaggle/input/enwiki-cirrus-20230701-e5-large-part*/*.npy")))
        elif "bge-base-15" in args.model_name:
            files_np = sorted(list(glob("/kaggle/input/cirruswiki512-bge-base-en-v1-5-part*/*.npy")))
        elif "gte-base" in args.model_name:
            files_np = sorted(list(glob("/kaggle/input/cirruswiki512-gte-base-part*/*.npy")))
        elif "bge-large" in args.model_name:
            files_np = sorted(list(glob("/kaggle/input/enwiki-cirrus-20230701-bge-large-part*/*.npy")))
        elif "gte-large" in args.model_name:
            files_np = sorted(list(glob("/kaggle/input/enwiki-cirrus-20230701-gte-large-part*/*.npy")))
    elif args.wiki == "new":
        if "gte-base-title" in args.model_name:
            files_np = sorted(list(glob("/kaggle/input/gte-base-title-part*/*.npy")))
        elif "e5-base-title" in args.model_name:
            files_np = sorted(list(glob("/kaggle/input/newwiki-e5-base-title-part*/*.npy")))
        elif "e5-base" in args.model_name:
            files_np = sorted(list(glob("/kaggle/input/newwiki-e5-base-part*/*.npy")))
        elif "gte-base" in args.model_name:
            files_np = sorted(list(glob("/kaggle/input/newwiki-gte-part*/*.npy")))
        elif "gte-large" in args.model_name:
            files_np = sorted(list(glob("/kaggle/input/wiki31m-gte-large-title-p*/*.npy")))
        elif "bge-base" in args.model_name:
            files_np = sorted(list(glob("/kaggle/input/wiki31m-bge-base-title-part*/*.npy")))
        elif "e5-large" in args.model_name:
            files_np = sorted(list(glob("/kaggle/input/wiki-30-08-e5-large-title-p*/*.npy")))
        elif "bge-large" in args.model_name:
            files_np = sorted(list(glob("/kaggle/input/wiki31m-bge-large-title-p*/*.npy")))
    elif args.wiki == "last":
        files_np = sorted(list(glob("/kaggle/input/enwiki-cirrus-256-e5-base-part*/*.npy")))       

    files_all = [(x, y) for x, y in zip(files_all, files_np)]
    files = [files_all[:len(files_all)//2], files_all[len(files_all)//2:]]


    if "e5-base" in args.model_name:
        model = SentenceTransformer('/kaggle/input/intfloat-e5-base-v2').to("cuda:0")
    elif "e5-large" in args.model_name:
        model = SentenceTransformer('/kaggle/input/intfloat-e5-large-v2').to("cuda:0")
    elif "bge-base-15" in args.model_name: 
        model = SentenceTransformer('/kaggle/input/baai-bge-base-en-v1-5').to("cuda:0")
    elif "gte-base" in args.model_name: 
        model = SentenceTransformer('/kaggle/input/thenlper-gte-base').to("cuda:0")
    elif "gte-large" in args.model_name:
        model = SentenceTransformer('/kaggle/input/thenlper-gte-large').to("cuda:0")
    elif "bge-base" in args.model_name:
        model = SentenceTransformer('/kaggle/input/baai-bge-base-en').to("cuda:0")
    elif "bge-large" in args.model_name:
        model = SentenceTransformer('/kaggle/input/baai-bge-large-en').to("cuda:0")

    test = pd.read_parquet("test_raw.pq")

    embs = []
    for idx, row in tqdm(test.iterrows(), total=len(test)):
        if "e5" in args.model_name:
            if args.ind == 1:
                sentences = [
                    "query: " + row.prompt + " " + row.A,
                    "query: " + row.prompt + " " + row.B,
                    "query: " + row.prompt + " " + row.C,
                    "query: " + row.prompt + " " + row.D,
                    "query: " + row.prompt + " " + row.E,
                ]
            else:
                sentences = ["query: " + row.prompt + " " + row.A + " " + row.B + " " + row.C + " " + row.D + " " + row.E]
        elif "bge" in args.model_name:
            if args.ind == 1:
                sentences = [
                    "Represent this sentence for searching relevant passages: " + row.prompt + " " + row.A,
                    "Represent this sentence for searching relevant passages: " + row.prompt + " " + row.B,
                    "Represent this sentence for searching relevant passages: " + row.prompt + " " + row.C,
                    "Represent this sentence for searching relevant passages: " + row.prompt + " " + row.D,
                    "Represent this sentence for searching relevant passages: " + row.prompt + " " + row.E,
                ]
            else:
                sentences = ["Represent this sentence for searching relevant passages: " + row.prompt + " " + row.A + " " + row.B + " " + row.C + " " + row.D + " " + row.E]    
        elif "gte" in args.model_name:
            if args.ind == 1:
                sentences = [
                    row.prompt + " " + row.A,
                    row.prompt + " " + row.B,
                    row.prompt + " " + row.C,
                    row.prompt + " " + row.D,
                    row.prompt + " " + row.E,
                ]
            else:
                sentences = [row.prompt + " " + row.A + " " + row.B + " " + row.C + " " + row.D + " " + row.E]

        embeddings = torch.Tensor(model.encode(sentences, show_progress_bar=False, normalize_embeddings=True))
        embs.append(torch.nn.functional.normalize(embeddings, dim=1))

    if args.ind == 1:
        query_embeddings = torch.Tensor(np.concatenate(embs)).squeeze(1)
    else:
        query_embeddings = torch.Tensor(np.stack(embs)).squeeze(1)
    print(query_embeddings.shape)

    # Create placeholders for top-k matches
    all_vals_gpu_0 = torch.full((len(test), TOP_K), -float('inf'), dtype=torch.float16)
    all_texts_gpu_0 = [[None] * TOP_K for _ in range(len(all_vals_gpu_0))]

    all_vals_gpu_1 = torch.full((len(test), TOP_K), -float('inf'), dtype=torch.float16)
    all_texts_gpu_1 = [[None] * TOP_K for _ in range(len(all_vals_gpu_1))]

    print(all_vals_gpu_0.shape)
    print(len(all_texts_gpu_0))


    def cos_similarity_matrix(a: torch.Tensor, b: torch.Tensor, eps: float = 1e-8):
        """Calculates cosine similarities between tensor a and b."""
        sim_mt = torch.mm(a, b.transpose(0, 1))
        return sim_mt


    def get_topk(embeddings_from, embeddings_to, topk=1000, bs=512):
        chunk = bs
        embeddings_chunks = embeddings_from.split(chunk)

        vals = []
        inds = []
        for idx in tqdm(range(len(embeddings_chunks))):
            cos_sim_chunk = cos_similarity_matrix(embeddings_chunks[idx].to(embeddings_to.device).half(), embeddings_to).float()

            cos_sim_chunk = torch.nan_to_num(cos_sim_chunk, nan=0.0)

            topk = min(topk, cos_sim_chunk.size(1))
            vals_chunk, inds_chunk = torch.topk(cos_sim_chunk, k=topk, dim=1)
            vals.append(vals_chunk[:, :].detach().cpu())
            inds.append(inds_chunk[:, :].detach().cpu())

            del vals_chunk
            del inds_chunk
            del cos_sim_chunk

        vals = torch.cat(vals).detach().cpu()
        inds = torch.cat(inds).detach().cpu()

        return inds, vals


    def insert_value_at(tensor, value, position):
        # Ensure the position is valid
        if position < 0 or position >= len(tensor):
            raise ValueError("Position should be between 0 and tensor length - 1.")

        # Slice the tensor into two parts
        left = tensor[:position]
        right = tensor[position:]

        # Create a tensor for the value to be inserted
        value_tensor = torch.tensor([value], dtype=tensor.dtype)

        # Concatenate the tensors together and slice to the original length
        result = torch.cat([left, value_tensor, right])[:-1]

        return result


    def insert_value_at_list(lst, value, position):
        # Ensure the position is valid
        if position < 0 or position >= len(lst):
            raise ValueError("Position should be between 0 and list length - 1.")

        # Insert value at the specified position
        lst.insert(position, value)

        # Remove the last value to maintain original length
        lst.pop()

        return lst
    
    from collections import OrderedDict

    def get_first_5_unique_strings(input_list):
#         print(len(input_list))
        unique_strings = list(OrderedDict.fromkeys(input_list))
#         print(unique_strings)
        while len(unique_strings) < args.topk:
            # TODO: check why this is ever happening
            unique_strings.append("No context found")
            print("appending None")
        return unique_strings[:args.topk]


    def load_data(files, device):
        for file, file_np in tqdm(files):
            df = pd.read_parquet(file, engine="pyarrow", use_threads=True)
            file_embeddings = np.load(file_np)

            data_embeddings = torch.Tensor(file_embeddings).to(device).half()
            data_embeddings = torch.nn.functional.normalize(data_embeddings, dim=1)

            max_inds, max_vals = get_topk(query_embeddings, data_embeddings, topk=TOP_K, bs=8)

            # loop through all queries (test)
            for i in range(len(test)):
                if args.ind == 1:
                    for answer in range(5):
                        for new in range(TOP_K):
                            if device == "cuda:0":
                                if max_vals[i * 5 + answer][new].item() < all_vals_gpu_0[i][TOP_K - 1]:
                                    break
                                for old in range(TOP_K):
                                    if max_vals[i * 5 + answer][new].item() > all_vals_gpu_0[i][old]:
                                        all_vals_gpu_0[i] = insert_value_at(all_vals_gpu_0[i], value=max_vals[i * 5 + answer][new].item(), position=old)
                                        all_texts_gpu_0[i] = insert_value_at_list(all_texts_gpu_0[i], value=df.iloc[max_inds[i * 5 + answer][new].item()].text, position=old)
                                        break
                            else:
                                if max_vals[i * 5 + answer][new].item() < all_vals_gpu_1[i][TOP_K - 1]:
                                    break
                                for old in range(TOP_K):
                                    if max_vals[i * 5 + answer][new].item() > all_vals_gpu_1[i][old]:
                                        all_vals_gpu_1[i] = insert_value_at(all_vals_gpu_1[i], value=max_vals[i * 5 + answer][new].item(), position=old)
                                        all_texts_gpu_1[i] = insert_value_at_list(all_texts_gpu_1[i], value=df.iloc[max_inds[i * 5 + answer][new].item()].text, position=old)
                                        break
                else:
                    # start with highest new val (pos 0) vs worst value already in the toplist (pos topk - 1)
                    for new in range(TOP_K):
                        if device == "cuda:0":
                            if max_vals[i][new].item() < all_vals_gpu_0[i][TOP_K - 1]:
                                break
                            for old in range(TOP_K):
                                if max_vals[i][new].item() > all_vals_gpu_0[i][old]:
                                    all_vals_gpu_0[i] = insert_value_at(all_vals_gpu_0[i], value=max_vals[i][new].item(), position=old)
                                    all_texts_gpu_0[i] = insert_value_at_list(all_texts_gpu_0[i], value=df.iloc[max_inds[i][new].item()].text, position=old)
                                    break
                        else:
                            if max_vals[i][new].item() < all_vals_gpu_1[i][TOP_K - 1]:
                                break
                            for old in range(TOP_K):
                                if max_vals[i][new].item() > all_vals_gpu_1[i][old]:
                                    all_vals_gpu_1[i] = insert_value_at(all_vals_gpu_1[i], value=max_vals[i][new].item(), position=old)
                                    all_texts_gpu_1[i] = insert_value_at_list(all_texts_gpu_1[i], value=df.iloc[max_inds[i][new].item()].text, position=old)
                                    break

            if not os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
                break


    Parallel(n_jobs=2, backend="threading")(delayed(load_data)(files[i], f"cuda:{i}") for i in range(2))
    all_vals = torch.hstack([all_vals_gpu_0, all_vals_gpu_1])
    val, inds = torch.topk(all_vals.float(), axis=1, k=TOP_K)
    all_texts = [[(t0 + t1)[inner_idx.item()] for inner_idx in idx] for t0, t1, idx in zip(all_texts_gpu_0, all_texts_gpu_1, inds)]
    # all_vals = [[(list(t0) + list(t1))[inner_idx.item()] for inner_idx in idx] for t0, t1, idx in zip(all_vals_gpu_0, all_vals_gpu_1, inds)]


    def remove_consecutive_duplicates(input_list):
        if not input_list:
            return [' '] * args.topk

        new_list = [input_list[0]]
        for i in range(1, len(input_list)):
            if input_list[i] != input_list[i-1]:
                new_list.append(input_list[i])

        # Append empty strings if new_list length is less than 5
        while len(new_list) < args.topk:
            new_list.append(' ')

        return new_list
    
    if args.ind == 1:
        all_texts = [get_first_5_unique_strings(x) for x in all_texts]
    else:
        all_texts = [remove_consecutive_duplicates(lst) for lst in all_texts]

    test["context"] = ["\n###\n".join([x[i] for i in list(range(args.topk))[::-1]]) for x in all_texts]

    test["context_v2"] = ["Context 4: " + x[4] + "\n###\n" + "Context 3: " + x[3] + "\n###\n" + "Context 2: " + x[2] + "\n###\n" + "Context 1: " + x[1] + "\n###\n" + "Context 0: " + x[0] for x in all_texts]

    test["context_deberta"] = [" [SEP] ".join(x[:args.topk]) for x in all_texts]

    print(test["context"].values[0])
    print("\nV2:\n")
    print(test["context_v2"].values[0])
    print("\nDEBERTA:\n")
    print(test["context_deberta"].values[0])

    test.to_parquet(f"{args.test_file}.pq", index=False)

Writing get_topk.py


In [7]:
!python get_topk.py --wiki "cirrus" --model_name "e5-large" --test_file "test" --topk 5 --ind 0

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Using cirrus wiki
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unab

In [8]:
!python get_topk.py --wiki "new" --model_name "e5-base-title" --test_file "test2" --topk 5 --ind 0

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Using new wiki
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable 

In [9]:
!python get_topk.py --wiki "new" --model_name "gte-large" --test_file "test3" --topk 5 --ind 0

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Using new wiki
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable 

In [10]:
!python get_topk.py --wiki "cirrus" --model_name "gte-base" --test_file "test4" --topk 3 --ind 0

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Using cirrus wiki
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unab

In [11]:
!python get_topk.py --wiki "cirrus" --model_name "e5-large" --test_file "test5" --topk 5 --ind 0

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Using cirrus wiki
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unab

In [12]:
!python get_topk.py --wiki "cirrus" --model_name "bge-large" --test_file "test6" --topk 5 --ind 1

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Using cirrus wiki
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unab

In [13]:
import gc
gc.collect()

0

In [14]:
!pip install /kaggle/input/sci-llm-pip-v2/bitsandbytes-0.41.0-py3-none-any.whl
!pip install --no-index --find-links="/kaggle/input/transformers-main" /kaggle/input/transformers-main/transformers-4.34.0-py3-none-any.whl
!pip install /kaggle/input/sci-llm-pip-v2/tiktoken-0.4.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl

Processing /kaggle/input/sci-llm-pip-v2/bitsandbytes-0.41.0-py3-none-any.whl
Looking in links: /kaggle/input/transformers-main
Processing /kaggle/input/transformers-main/transformers-4.34.0-py3-none-any.whl
Processing /kaggle/input/transformers-main/tokenizers-0.14.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (from transformers==4.34.0)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.30.2
    Uninstalling transformers-4.30.2:
      Successfully uninstalled transformers-4.30.2
Processing /kaggle/input/sci-llm-pip-v2/tiktoken-0.4.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl


In [15]:
os.environ["TIKTOKEN_CACHE_DIR"] = "/kaggle/input/tiktoken-cache"

In [16]:
%%writefile inference.py


from transformers import AutoModelForCausalLM, AutoTokenizer
import itertools
import random
import argparse
import os
import pandas as pd
import numpy as np
from string import Template
from pathlib import Path
import time
import torch
from tqdm.auto import tqdm
import gc
import itertools

#os.environ["TOKENIZERS_PARALLELISM"] = "false"

def precision_at_k(r, k):
    """Precision at k"""
    assert k <= len(r)
    assert k != 0
    return sum(int(x) for x in r[:k]) / k

def MAP_at_3(predictions, true_items):
    """Score is mean average precision at 3"""
    U = len(predictions)
    map_at_3 = 0.0
    for u in range(U):
        user_preds = predictions[u]
        user_true = true_items[u]
        user_results = [1 if item == user_true else 0 for item in user_preds]
        for k in range(min(len(user_preds), 3)):
            map_at_3 += precision_at_k(user_results, k+1) * user_results[k]
    return map_at_3 / U

def longest_common_prefix(strs):
    if not strs:
        return ""

    shortest = min(strs,key=len)

    for i, char in enumerate(shortest):
        for other in strs:
            if other[i] != char:
                return shortest[:i]

    return shortest 

if __name__ == "__main__":
    
    ap = argparse.ArgumentParser()
    ap.add_argument('--device', type=str, required=True)
    ap.add_argument('--model_name', type=str, required=True)
    ap.add_argument('--quantization', type=int, required=True)
    ap.add_argument('--model_type', type=int, required=True)
    ap.add_argument('--test_file', type=str, required=True)
    args = ap.parse_args()
    
    if args.device == "auto":
        DEVICE_MAP = "auto"
        DEVICE = "cuda:0"
    else:
        DEVICE_MAP = {"":args.device}
        DEVICE = args.device
    
    llm_backbone = args.model_name
    
    test = pd.read_parquet(args.test_file).reset_index(drop=True)


    new_obs = []
    for idx, row in test.iterrows():
        for opt in "ABCDE":
            new_obs.append((row["id"], row["context"], row["context_v2"], row["prompt"], row[opt]))
    df = pd.DataFrame(new_obs, columns=["id", "context", "context_v2", "question", "answer"])

    
    print(df.head(1))
    

    tokenizer = AutoTokenizer.from_pretrained(
        llm_backbone,
        use_fast=True,
        trust_remote_code=True,
        padding_side="right",
        truncation_side="left"
    )
    
    if tokenizer.pad_token is None:
        if tokenizer.unk_token is not None:
            tokenizer.pad_token = tokenizer.unk_token
        else:
            tokenizer.pad_token = tokenizer.eos_token
    
    from transformers import BitsAndBytesConfig
    
    
#     quantization_config = BitsAndBytesConfig(
#        load_in_4bit=True,
#        bnb_4bit_quant_type="nf4",
#        bnb_4bit_use_double_quant=False,
#        bnb_4bit_compute_dtype=torch.float16
#     )

    if args.quantization == 0:
        quantization_config = None

    elif args.quantization == 1:
        quantization_config = BitsAndBytesConfig(
            load_in_8bit=True,
            llm_int8_threshold=0.0,
        )


    model = AutoModelForCausalLM.from_pretrained(
        llm_backbone,
        torch_dtype=torch.float16,
        quantization_config=quantization_config,
        device_map=DEVICE_MAP,
        low_cpu_mem_usage=True,
        trust_remote_code=True
    ).eval()

    head_weights = torch.load(llm_backbone + "/head.pth", map_location="cpu")
    hidden_size = head_weights.shape[1]
    print(hidden_size)
    # hidden_size = 32000
    head = torch.nn.Linear(hidden_size, 1, bias=False)

    head.weight.data = head_weights

    head.to(DEVICE).eval()

    model.config.pad_token_id = tokenizer.pad_token_id
    
    gc.collect()


    

    import itertools
    import random
    maps = []

    progress_bar = tqdm(df.iterrows(), total=len(df))

    preds = []
    instructions = []
    pooled = []
    past_key_values = None
    for idx,row in progress_bar:
        
        inst = f"Answer: {row['answer']}\n###\nIs this answer correct? "
        #inst = "a b c d"*10_000
        instructions.append(inst)
        
        if idx % 5 == 0:
            
            if past_key_values is not None:
                del past_key_values
            
            preprompt = f"{row['context_v2']}\n###\nQuestion: {row['question']}\n###\n"
            #preprompt = "a b c d"*10_000
            inputs = tokenizer(preprompt, return_tensors='pt', add_special_tokens=False, truncation=True, padding="longest", max_length=1024)
            
            tok_length = inputs["input_ids"].shape[1] + tokenizer(instructions, return_tensors='pt', add_special_tokens=False, truncation=True, padding="longest", max_length=1024)["input_ids"].shape[1]
            
            BATCH_SIZE = 5
#             if tok_length >= 1024:
#                 BATCH_SIZE = 1
            #print(BATCH_SIZE)
            
            with torch.no_grad():
                past_key_values = list(model(input_ids=inputs["input_ids"].to(DEVICE)).past_key_values)
                
                for idx0 in range(len(past_key_values)):
                    past_key_values[idx0] = list(past_key_values[idx0])
                    for idx1 in range(len(past_key_values[idx0])):
                        past_key_values[idx0][idx1] = past_key_values[idx0][idx1].expand(BATCH_SIZE,-1,-1,-1)
            del inputs
                        
        

        if (idx+1) % BATCH_SIZE == 0 or idx == len(df) - 1:
        #if True:

            inputs = tokenizer(instructions, return_tensors='pt', add_special_tokens=False, truncation=True, padding="longest")

            with torch.no_grad():
                out = model(input_ids=inputs["input_ids"].to(DEVICE), past_key_values=past_key_values).logits
                
                for jjj in range(len(out)):
                    att_idx = inputs["attention_mask"].sum(dim=1)[jjj]-1
                    pooled.append(out[jjj,att_idx,:].float().unsqueeze(0))
                    
            instructions = []
            del out
            del inputs
         
        if (idx+1) % 5 == 0:
            with torch.no_grad():
                pooled = torch.cat(pooled)
                
                if args.model_type == 2:
                    new_poolings = pooled.reshape(1,-1)
                    new_poolings = new_poolings.reshape(5, new_poolings.size(-1)//5)
                    permutations = list(itertools.permutations(range(new_poolings.size(0))))
                    tta_logits = torch.zeros((5,1)).to(new_poolings.device)
                    for perm in permutations:
                        permuted_poolings = new_poolings[torch.tensor(perm)]
                        o = head(permuted_poolings.float().reshape(1,-1))
                        tta_logits[torch.tensor(perm)] += o.reshape(-1,1)
                    logits = tta_logits / len(permutations)
                    logits = logits[:,0]
                    logits = logits.detach().cpu().numpy()
                else:
                    new_poolings = []

                    indexes = np.arange(0, 5)
                    for jj in indexes:
                        if args.model_type == 1:
                            other_embeddings = pooled[[jjj for jjj in indexes if jjj != jj]]
                            new_poolings.append(torch.cat([pooled[jj], torch.mean(other_embeddings, dim=0)]))
                        else:
                            new_poolings.append(pooled[jj])
                    new_poolings = torch.stack(new_poolings)
                    
                    logits = head(new_poolings)
                    logits = logits[:,0]
                    logits = logits.detach().cpu().numpy()

                for lg in logits:
                    preds.append(lg)

            del logits
            pooled = []

    print(inst)
    np.save(f"scores_{llm_backbone.split('/')[-1]}_{args.test_file[:-3]}", preds)

Writing inference.py


In [17]:
%%writefile run.sh

python inference.py --device "auto" --model_name "/kaggle/input/teamhydrogen-cool-giraffe-prompt-2409" --quantization 0 --model_type 1 --test_file "test3.pq"  &&
python inference.py --device "auto" --model_name "/kaggle/input/teamhydrogen-white-malamute-prompt-openorca-v2" --quantization 0 --model_type 1 --test_file "test.pq"  &&
python inference.py --device "auto" --model_name "/kaggle/input/teamhydrogen-raspberry-beaver-1" --quantization 0 --model_type 1 --test_file "test5.pq"  &&
python inference.py --device "auto" --model_name "/kaggle/input/teamhydrogen-cool-giraffe-prompt-0710" --quantization 0 --model_type 1 --test_file "test6.pq"  &&

wait 
echo "All done"

Writing run.sh


In [18]:
!sh run.sh

   id  ...                                             answer
0   0  ...  RNA-templated transcription is the method of t...

[1 rows x 5 columns]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Using pad_token, but it is not set yet.
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tens

In [19]:
test = pd.read_parquet("test.pq")

In [20]:
all_scores = []

In [21]:
%%writefile inference_2.py


from transformers import AutoModelForCausalLM, AutoTokenizer
import itertools
import random
import argparse
import os
import pandas as pd
import numpy as np
from string import Template
from pathlib import Path
import time
import torch
from tqdm.auto import tqdm
import gc
import itertools

#os.environ["TOKENIZERS_PARALLELISM"] = "false"

def precision_at_k(r, k):
    """Precision at k"""
    assert k <= len(r)
    assert k != 0
    return sum(int(x) for x in r[:k]) / k

def MAP_at_3(predictions, true_items):
    """Score is mean average precision at 3"""
    U = len(predictions)
    map_at_3 = 0.0
    for u in range(U):
        user_preds = predictions[u]
        user_true = true_items[u]
        user_results = [1 if item == user_true else 0 for item in user_preds]
        for k in range(min(len(user_preds), 3)):
            map_at_3 += precision_at_k(user_results, k+1) * user_results[k]
    return map_at_3 / U

def longest_common_prefix(strs):
    if not strs:
        return ""

    shortest = min(strs,key=len)

    for i, char in enumerate(shortest):
        for other in strs:
            if other[i] != char:
                return shortest[:i]

    return shortest 

if __name__ == "__main__":
    
    ap = argparse.ArgumentParser()
    ap.add_argument('--device', type=str, required=True)
    ap.add_argument('--model_name', type=str, required=True)
    ap.add_argument('--quantization', type=int, required=True)
    ap.add_argument('--model_type', type=int, required=True)
    ap.add_argument('--test_file', type=str, required=True)
    args = ap.parse_args()
    
    if args.device == "auto":
        DEVICE_MAP = "auto"
        DEVICE = "cuda:0"
    else:
        DEVICE_MAP = {"":args.device}
        DEVICE = args.device
    
    llm_backbone = args.model_name
    
    test = pd.read_parquet(args.test_file).reset_index(drop=True)


    new_obs = []
    for idx, row in test.iterrows():
        for opt in "ABCDE":
            new_obs.append((row["id"], row["context"], row["prompt"], row[opt]))
    df = pd.DataFrame(new_obs, columns=["id", "context", "question", "answer"])

    
    print(df.head(1))
    

    tokenizer = AutoTokenizer.from_pretrained(
        llm_backbone,
        use_fast=True,
        trust_remote_code=True,
        padding_side="right",
        truncation_side="left"
    )
    
    if tokenizer.pad_token is None:
        if tokenizer.unk_token is not None:
            tokenizer.pad_token = tokenizer.unk_token
        else:
            tokenizer.pad_token = tokenizer.eos_token
    
    from transformers import BitsAndBytesConfig
    
    
#     quantization_config = BitsAndBytesConfig(
#        load_in_4bit=True,
#        bnb_4bit_quant_type="nf4",
#        bnb_4bit_use_double_quant=False,
#        bnb_4bit_compute_dtype=torch.float16
#     )

    if args.quantization == 0:
        quantization_config = None

    elif args.quantization == 1:
        quantization_config = BitsAndBytesConfig(
            load_in_8bit=True,
            llm_int8_threshold=0.0,
        )


    model = AutoModelForCausalLM.from_pretrained(
        llm_backbone,
        torch_dtype=torch.float16,
        quantization_config=quantization_config,
        device_map=DEVICE_MAP,
        low_cpu_mem_usage=True,
        trust_remote_code=True
    ).eval()

    head_weights = torch.load(llm_backbone + "/head.pth", map_location="cpu")
    hidden_size = head_weights.shape[1]
    print(hidden_size)
    # hidden_size = 32000
    head = torch.nn.Linear(hidden_size, 1, bias=False)

    head.weight.data = head_weights

    head.to(DEVICE).eval()

    model.config.pad_token_id = tokenizer.pad_token_id
    
    gc.collect()


    

    import itertools
    import random
    maps = []

    progress_bar = tqdm(df.iterrows(), total=len(df))

    preds = []
    instructions = []
    pooled = []
    past_key_values = None
    for idx,row in progress_bar:
        
        inst = f"{row['answer']}"
        #inst = "a b c d"*10_000
        instructions.append(inst)
        
        if idx % 5 == 0:
            
            if past_key_values is not None:
                del past_key_values
            
            preprompt = f"{row['context']}\n###\n{row['question']}\n###\n"
            #preprompt = "a b c d"*10_000
            inputs = tokenizer(preprompt, return_tensors='pt', add_special_tokens=False, truncation=True, padding="longest", max_length=1024)
            
            tok_length = inputs["input_ids"].shape[1] + tokenizer(instructions, return_tensors='pt', add_special_tokens=False, truncation=True, padding="longest", max_length=1024)["input_ids"].shape[1]
            
            BATCH_SIZE = 5
#             if tok_length >= 1024:
#                 BATCH_SIZE = 1
            #print(BATCH_SIZE)
            
            with torch.no_grad():
                past_key_values = list(model(input_ids=inputs["input_ids"].to(DEVICE)).past_key_values)
                
                for idx0 in range(len(past_key_values)):
                    past_key_values[idx0] = list(past_key_values[idx0])
                    for idx1 in range(len(past_key_values[idx0])):
                        past_key_values[idx0][idx1] = past_key_values[idx0][idx1].expand(BATCH_SIZE,-1,-1,-1)
            del inputs
                        
        

        if (idx+1) % BATCH_SIZE == 0 or idx == len(df) - 1:
        #if True:

            inputs = tokenizer(instructions, return_tensors='pt', add_special_tokens=False, truncation=True, padding="longest")

            with torch.no_grad():
                out = model(input_ids=inputs["input_ids"].to(DEVICE), past_key_values=past_key_values).logits
                
                for jjj in range(len(out)):
                    att_idx = inputs["attention_mask"].sum(dim=1)[jjj]-1
                    pooled.append(out[jjj,att_idx,:].float().unsqueeze(0))
                    
            instructions = []
            del out
            del inputs
         
        if (idx+1) % 5 == 0:
            with torch.no_grad():
                pooled = torch.cat(pooled)
                
                if args.model_type == 2:
                    new_poolings = pooled.reshape(1,-1)
                    new_poolings = new_poolings.reshape(5, new_poolings.size(-1)//5)
                    permutations = list(itertools.permutations(range(new_poolings.size(0))))
                    tta_logits = torch.zeros((5,1)).to(new_poolings.device)
                    for perm in permutations:
                        permuted_poolings = new_poolings[torch.tensor(perm)]
                        o = head(permuted_poolings.float().reshape(1,-1))
                        tta_logits[torch.tensor(perm)] += o.reshape(-1,1)
                    logits = tta_logits / len(permutations)
                    logits = logits[:,0]
                    logits = logits.detach().cpu().numpy()
                else:
                    new_poolings = []

                    indexes = np.arange(0, 5)
                    for jj in indexes:
                        if args.model_type == 1:
                            other_embeddings = pooled[[jjj for jjj in indexes if jjj != jj]]
                            new_poolings.append(torch.cat([pooled[jj], torch.mean(other_embeddings, dim=0)]))
                        else:
                            new_poolings.append(pooled[jj])
                    new_poolings = torch.stack(new_poolings)
                    
                    logits = head(new_poolings)
                    logits = logits[:,0]
                    logits = logits.detach().cpu().numpy()

                for lg in logits:
                    preds.append(lg)

            del logits
            pooled = []

    print(inst)
    np.save(f"scores_{llm_backbone.split('/')[-1]}_{args.test_file[:-3]}", preds)

Writing inference_2.py


In [22]:
%%writefile run.sh

python inference_2.py --device "auto" --model_name "/kaggle/input/teamhydrogen-nostalgic-mammoth-avg" --quantization 0 --model_type 1 --test_file "test4.pq"  &&

wait 
echo "All done"

Overwriting run.sh


In [23]:
!sh run.sh

   id  ...                                             answer
0   0  ...  RNA-templated transcription is the method of t...

[1 rows x 4 columns]
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/co

In [24]:
%%writefile run.sh

python inference_2.py --device "auto" --model_name "/kaggle/input/teamhydrogen-wealthy-ant-llama2-13b-1909" --quantization 1 --model_type 0 --test_file "test2.pq"  &&

wait 
echo "All done"

Overwriting run.sh


In [25]:
!sh run.sh

   id  ...                                             answer
0   0  ...  RNA-templated transcription is the method of t...

[1 rows x 4 columns]
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/co

In [26]:
import glob
from scipy.special import softmax

curr_scores = []

for f in glob.glob("scores_*.npy"):
    print(f)
    a = np.array(np.load(f))
    a = softmax(a.reshape(-1,5), axis=1)
    a = a.flatten()
    print(a.mean(axis=0))

    curr_scores.append(a)
    os.remove(f)
curr_scores = np.array(curr_scores)
all_scores.append(np.nanmean(curr_scores, axis=0))

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


scores_teamhydrogen-raspberry-beaver-1_test5.npy
0.2
scores_teamhydrogen-wealthy-ant-llama2-13b-1909_test2.npy
0.2
scores_teamhydrogen-white-malamute-prompt-openorca-v2_test.npy
0.2
scores_teamhydrogen-cool-giraffe-prompt-2409_test3.npy
0.2
scores_teamhydrogen-cool-giraffe-prompt-0710_test6.npy
0.2
scores_teamhydrogen-nostalgic-mammoth-avg_test4.npy
0.2


In [27]:
all_scores = np.nanmean(all_scores, axis=0)

In [28]:
all_scores.shape

(1500,)

In [29]:
preds = all_scores
preds.shape

(1500,)

In [30]:
preds = np.array(preds)
preds.shape

(1500,)

In [31]:
# We'll create a dictionary to convert option names (A, B, C, D, E) into indices and back again
options = 'ABCDE'
indices = list(range(5))

option_to_index = {option: index for option, index in zip(options, indices)}
index_to_option = {index: option for option, index in zip(options, indices)}
post_proc_preds = []

for gr in range(0, len(preds), 5):
    pr = preds[gr : gr + 5]
    pr = np.argsort(-pr)[:3]
    post_proc_preds.append(" ".join([index_to_option[x] for x in pr]))
len(post_proc_preds)

300

In [32]:
test['prediction'] = post_proc_preds

In [33]:
if CALC_SCORE:

    import numpy as np
    def precision_at_k(r, k):
        """Precision at k"""
        assert k <= len(r)
        assert k != 0
        return sum(int(x) for x in r[:k]) / k

    def MAP_at_3(predictions, true_items):
        """Score is mean average precision at 3"""
        U = len(predictions)
        map_at_3 = 0.0
        for u in range(U):
            user_preds = predictions[u]
            user_true = true_items[u]
            user_results = [1 if item == user_true else 0 for item in user_preds]
            for k in range(min(len(user_preds), 3)):
                map_at_3 += precision_at_k(user_results, k+1) * user_results[k]
        return map_at_3 / U

    maps = []
    for idx, row in test.iterrows():

        tl = row["answer"]
        pr = row["prediction"].split(" ")

        map = MAP_at_3([pr], [tl])
        print(map, tl, pr)
        maps.append(map)

    print(np.mean(maps))

0.5 D ['A', 'D', 'B']
1.0 D ['D', 'C', 'B']
0.0 A ['C', 'B', 'E']
0.0 B ['C', 'E', 'A']
1.0 A ['A', 'D', 'C']
0.5 B ['D', 'B', 'E']
1.0 B ['B', 'D', 'E']
0.0 A ['D', 'C', 'B']
0.0 E ['A', 'C', 'D']
0.5 A ['D', 'A', 'C']
1.0 B ['B', 'D', 'A']
0.5 D ['C', 'D', 'E']
1.0 D ['D', 'A', 'B']
1.0 D ['D', 'C', 'B']
1.0 A ['A', 'B', 'E']
1.0 D ['D', 'B', 'A']
1.0 D ['D', 'E', 'C']
0.5 C ['D', 'C', 'E']
1.0 A ['A', 'B', 'C']
1.0 B ['B', 'D', 'E']
0.3333333333333333 A ['E', 'B', 'A']
1.0 D ['D', 'C', 'B']
0.5 D ['A', 'D', 'E']
0.5 A ['D', 'A', 'E']
1.0 B ['B', 'D', 'A']
0.5 C ['E', 'C', 'A']
0.5 C ['A', 'C', 'D']
0.3333333333333333 D ['C', 'B', 'D']
1.0 E ['E', 'D', 'C']
0.5 D ['B', 'D', 'A']
0.3333333333333333 E ['C', 'B', 'E']
1.0 E ['E', 'B', 'D']
0.0 B ['C', 'A', 'E']
1.0 D ['D', 'C', 'B']
1.0 D ['D', 'A', 'C']
0.0 B ['D', 'A', 'C']
1.0 D ['D', 'B', 'A']
1.0 C ['C', 'A', 'D']
1.0 E ['E', 'C', 'D']
1.0 C ['C', 'B', 'A']
0.3333333333333333 B ['E', 'D', 'B']
1.0 D ['D', 'B', 'E']
1.0 A ['A', 'B',

In [34]:
# Once we write our submission file we're good to submit!
test[['id', 'prediction']].to_csv('submission.csv', index=False)
pd.read_csv("submission.csv")

,id,prediction
0,0,A D B
1,1,D C B
2,2,C B E
3,3,C E A
4,4,A D C
...,...,...
295,295,A D C
296,296,B A C
297,297,D C B
298,298,A E D
